In [ ]:
import requests
import json
import pandas as pd
import datetime
import sqlite3

## Load in Raw Data

In [ ]:
db_location = '../data/meh_scraper_qa3.db'

In [ ]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [ ]:
raw_data = pd.read_sql_query("select * from raw_response_backup"
                             , con)

In [ ]:
raw_data.shape

In [ ]:
raw_data.head()

### Remove Initial Dups from raw

In [ ]:
raw_backup_dedup = raw_data.drop_duplicates(['date'])
raw_backup_dedup.shape

## Peak at Products

In [ ]:
products_previous = pd.read_sql_query("select * from products"
                             , con)

In [ ]:
products_previous.shape

In [ ]:
products_previous.head()

## Begin Fix 

In [ ]:
current_dt = datetime.datetime.now()
current_date = str(current_dt.date())
current_time = str(current_dt.time())
print(current_dt)

In [ ]:
fake_time = datetime.time(0, 0, 1)

In [ ]:
def flatten_dict(parsed_site_series, k1):
    final_dict = {}
    for k,v in parsed_site_series.items():
        if k1 !='':
            new_k = k1 + '_' + k
        else:
            new_k = k

        if type(v) in [str, int, float]:
            final_dict[new_k] = v
        elif type(v) == dict:
            temp = flatten_dict(v, new_k)
            final_dict.update(temp)
        else:
            print(k + ' is ' + str(type(v)))
            pass
    return(final_dict)            

In [ ]:
def parse_raw_row(raw_data_row):
    # Parse data from raw response
    raw_resp = raw_data_row['raw_response']
    date = raw_data_row['date']
    site_data = json.loads(raw_resp)
    parsed = flatten_dict(site_data, '')

    # Add time info 
    fake_time = datetime.time(0, 0, 1)
    parsed['time'] = fake_time
    parsed['date'] = date

    parsed_series = pd.DataFrame(parsed, index=[1])
    return(parsed_series)

In [ ]:
raw_data_to_parse = raw_data.drop_duplicates(['date'])

In [ ]:
complete_rescore = raw_data_to_parse.apply(lambda x: parse_raw_row(x), axis=1, )

In [ ]:
complete_rescore = pd.concat(complete_rescore.values)

In [ ]:
complete_rescore

## Validations

In [ ]:
poll_cols = [x for x in complete_rescore.columns if 'poll' in x.lower()]
assert(len(poll_cols) > 3)
assert('poll_id' in poll_cols)

poll_cols

In [ ]:
assert(raw_backup_dedup['date'].nunique() == raw_backup_dedup.shape[0])

In [ ]:
con.close()

## Update Prod: CAREFUL!!!  
Use QA, check result (run through this notebook) before coming back to remove qa from db link 

In [ ]:
db_location_PROD = '../data/meh_scraper_qa3.db'
con_PROD = sqlite3.connect(db_location_PROD)
cursor_PROD = con_PROD.cursor()

In [ ]:
complete_rescore.to_sql('products', con_PROD, if_exists='replace', index=False, )

In [ ]:
raw_backup_dedup.to_sql('raw_response_backup', con_PROD, if_exists='replace', index=False)

In [ ]:
con_PROD.commit()
cursor_PROD.close()